In [2]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import pickle

sys.path.append("..")
from dm21cm.dm_params import DMParams

sys.path.append(os.environ['DH_DIR'])
from darkhistory.main import evolve as evolve_DH
from darkhistory.config import load_data

In [3]:
tf_version = 'zf01'

load_data('dep_tf', prefix=f"{os.environ['DH_DATA_DIR']}/../DHdata_v1_1_{tf_version}")
load_data('ics_tf', prefix=f"{os.environ['DH_DATA_DIR']}/../DHdata_v1_1_{tf_version}");

****** Loading transfer functions... ******
Using data at /n/holyscratch01/iaifi_lab/yitians/darkhistory/DHdata_v1_1/../DHdata_v1_1_zf01
    for propagating photons...  

 Done!
    for low-energy photons...  Done!
    for low-energy electrons...  Done!
    for high-energy deposition...  Done!
    for total upscattered CMB energy rate...  Done!
****** Loading complete! ******
****** Loading transfer functions... ******
    for inverse Compton (Thomson)...  Done!
    for inverse Compton (relativistic)...  Done!
    for inverse Compton (energy loss)...  Done!
****** Loading complete! ******


In [5]:
# phph
dm_params = DMParams(
    mode = 'decay',
    primary = 'phot_delta',
    m_DM = 1e8, # [eV]
    lifetime = 1e25, #[s]
)

In [4]:
# off
dm_params = DMParams(
    mode = 'decay',
    primary = 'phot_delta',
    m_DM = 1e8, # [eV]
    lifetime = 1e50, #[s]
)

In [11]:
run_name = 'xc_xrayST_noHe'

soln = evolve_DH(
    DM_process=dm_params.mode, mDM=dm_params.m_DM,
    primary=dm_params.primary,
    sigmav=dm_params.sigmav, lifetime=dm_params.lifetime,
    struct_boost=dm_params.struct_boost,
    start_rs=3000, end_rs=4.+1, coarsen_factor=12, verbose=1,
    # use fake reionization to get case-A recomb. coeff.
    reion_switch=True, reion_rs=47.,
    photoion_rate_func=[lambda x: 0., lambda x: 0., lambda x: 0.],
    photoheat_rate_func=[lambda x: 0., lambda x: 0., lambda x: 0.],
    cross_check_21cmfast=True,
    debug_no_bath=False,
    debug_inject_ST_xray=True,
)

pickle.dump(soln, open(f"{os.environ['DM21CM_DIR']}/outputs/dh/{run_name}_soln.p", 'wb'))

Loading time: 0.000 s


  0%|          | 0/534 [00:00<?, ?it/s]WARNING:root:Cross checking 21cmfast!


Initialization time: 0.027 s


  0%|          | 2/534 [00:00<00:54,  9.76it/s]

 lsoda--  at start of problem, too much accuracy     
       requested for precision of machine..  see tolsf (=r1) 
      in above message,  r1 =                  NaN


...

xrayST: injecting 5.514e-02 eV/Bavg
xrayST: injecting 5.800e-02 eV/Bavg
xrayST: injecting 6.115e-02 eV/Bavg
xrayST: injecting 6.433e-02 eV/Bavg
xrayST: injecting 6.755e-02 eV/Bavg
xrayST: injecting 7.081e-02 eV/Bavg
xrayST: injecting 7.411e-02 eV/Bavg
xrayST: injecting 7.744e-02 eV/Bavg


564it [01:00, 36.90it/s]

xrayST: injecting 8.114e-02 eV/Bavg
xrayST: injecting 8.504e-02 eV/Bavg
xrayST: injecting 8.898e-02 eV/Bavg
xrayST: injecting 9.295e-02 eV/Bavg
xrayST: injecting 9.696e-02 eV/Bavg
xrayST: injecting 1.010e-01 eV/Bavg
xrayST: injecting 1.051e-01 eV/Bavg
xrayST: injecting 1.095e-01 eV/Bavg


573it [01:00,  9.41it/s]

xrayST: injecting 1.142e-01 eV/Bavg
xrayST: injecting 1.190e-01 eV/Bavg
xrayST: injecting 1.238e-01 eV/Bavg
xrayST: injecting 1.286e-01 eV/Bavg
xrayST: injecting 1.335e-01 eV/Bavg
xrayST: injecting 1.384e-01 eV/Bavg
xrayST: injecting 1.433e-01 eV/Bavg
Main loop time: 60.873 s
